In [1]:
pip install llama-stack

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.7/634.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 19.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 20.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114330 sha256=0cdbdef2f1f302bebf72f14411dc01d740e58642c3d97

In [1]:
import pandas as pd

In [3]:
!pip install transformers accelerate datasets
!pip install xlsxwriter  # 如果要写 Excel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 32.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 34.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
#!pip install xlrd
!pip install openpyxl



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import pandas as pd

# 修改为你本地实际的路径
csv_path = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/cad_questions_combined.csv"
#excel_path_1 = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/Dataset_Text.xls"
excel_path_2 = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/Dataset_multiple choices.xlsx"

# 读取文件
df_csv = pd.read_csv(csv_path)
#df_excel_1 = pd.read_excel(excel_path_1, engine="xlrd")

df_excel_2 = pd.read_excel(excel_path_2)

# 查看前几行
print("CSV first rows：")
print(df_csv.head())

print("\nExcel first rows：")
print(df_excel_2.head())




CSV first rows：
                                            question  \
0  An ill 16 days old baby girl is brought to the...   
1        Earliest lesion seen in atherosclerosis is:   
2  Flipped LDH indicating Myocardial Infarction i...   
3  A 19 year old female presents with pain in the...   
4  A 63 year old man presents with a triad of ang...   

                                                 exp  cop                 opa  \
0  In CHF pallor, dyspnoea, tachypnoea, tachycard...  1.0                 CHF   
1  Ans: A (Fatty streaks) Ref: Robbins Pathologic...  1.0       Fatty streaks   
2  When LDH-1 is greater than LDH-2, it is referr...  1.0        LDH-1> LDH-2   
3  Answer: c) Aspirin (SCHWARTZ 19TH ED, P-1525; ...  3.0  Atropine injection   
4  Even severe AS may exist for many years withou...  3.0     antral stenosis   

                            opb                 opc  \
0      Glycogen storage disease        Pericarditis   
1            Intimal thickening  Fibrinoid necrosi

In [12]:
import pandas as pd

# 读取数据
df_csv = pd.read_csv(csv_path)

# 映射关系
choice_map = {
    1.0: "A",
    2.0: "B",
    3.0: "C",
    4.0: "D"
}

# 替换 cop 列的数值
df_csv["cop"] = df_csv["cop"].map(choice_map)

# 查看替换后的结果（可选）
print(df_csv[["cop", "opa", "opb", "opc", "opd"]].head())

# 如果你后续还要用这个修改过的 df_csv 生成训练文本，可以直接继续使用 df_csv


  cop                 opa                           opb                 opc  \
0   A                 CHF      Glycogen storage disease        Pericarditis   
1   A       Fatty streaks            Intimal thickening  Fibrinoid necrosis   
2   A        LDH-1> LDH-2                 LDH-2 > LDH-1       LDH-4 > LDH-5   
3   C  Atropine injection  Levothyroxine administration             Aspirin   
4   C     antral stenosis       Tricuspid regurgitation       Aoic stenosis   

                                                 opd  
0  Aberrant left coronary aery arising from pulmo...  
1                                             Plaque  
2                                      LDH-5 > LDH-4  
3                     Increase iodine intake in food  
4                                 Aoic regurgitation  


In [13]:
import pandas as pd

# 读取你的数据
#df_csv = pd.read_csv("你的CSV路径")
df_excel = pd.read_excel(excel_path_2)

# 示例处理 CSV 数据
csv_prompts = []
for _, row in df_csv.iterrows():
    if row["choice_type"] == "single":
        question = row["question"]
        options = f"A. {row['opa']}\nB. {row['opb']}\nC. {row['opc']}\nD. {row['opd']}"
        answer = row["cop"]
        prompt = f"### Instruction:\n{question}\n\n{options}\n\n### Response:\n{answer}"
        csv_prompts.append(prompt)

# 示例处理 Excel 数据
excel_prompts = []
for _, row in df_excel.iterrows():
    question = row["Question"]
    options = f"A. {row['Option A']}\nB. {row['Option B']}\nC. {row['Option C']}\nD. {row['Option D']}"
    answer = row["Answer"]
    prompt = f"### Instruction:\n{question}\n\n{options}\n\n### Response:\n{answer}"
    excel_prompts.append(prompt)

# 合并所有 prompts 并保存
all_prompts = csv_prompts + excel_prompts
with open("train_data.txt", "w", encoding="utf-8") as f:
    for item in all_prompts:
        f.write(item + "\n\n")


In [14]:
# 打印前5个 CSV prompts
print("CSV Prompts:")
for prompt in csv_prompts[:5]:
    print(prompt)
    print("=" * 50)

# 打印前5个 Excel prompts
print("Excel Prompts:")
for prompt in excel_prompts[:5]:
    print(prompt)
    print("=" * 50)


CSV Prompts:
### Instruction:
Earliest lesion seen in atherosclerosis is:

A. Fatty streaks
B. Intimal thickening
C. Fibrinoid necrosis
D. Plaque

### Response:
A
### Instruction:
Flipped LDH indicating Myocardial Infarction is represented by:

A. LDH-1> LDH-2
B. LDH-2 > LDH-1
C. LDH-4 > LDH-5
D. LDH-5 > LDH-4

### Response:
A
### Instruction:
A 63 year old man presents with a triad of angina, syncope and congestive hea failure. Which of the following valvular hea lesion can be suspected-

A. antral stenosis
B. Tricuspid regurgitation
C. Aoic stenosis
D. Aoic regurgitation

### Response:
C
### Instruction:
In a patient with a type of familial dyslipidemias he presented with palmar Xanthomas and he ias at increased risk of atherosclerosis and CAD.In Lipid profile there was elevation of triacylglycerols and cholesterol.And the IDL and chylomicrons were elevated.What is the pathophysiology of the condition?

A. LDL deficiency
B. VLDL overproduction
C. Apo C-2 deficiency
D. Apo E deficienc

In [ ]:
#转成huggingface格式后
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_path = "meta-llama/Llama-3-8b-Instruct"  # 或者你本地的转换好路径

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 随便选一个 prompt（记得替换）
prompt = "### Instruction:\nWhat is the earliest lesion in atherosclerosis?\nA. Fatty streaks\nB. Intimal thickening\nC. Fibrinoid necrosis\nD. Plaque\n\n### Response:"
output = qa_pipeline(prompt, max_new_tokens=50, do_sample=False)
print(output[0]['generated_text'])
